In [2]:
from pathlib import Path

import pandas as pd
import requests
import django
import json

django.setup()

In [2]:
from django.conf import settings
from django.db.models import F

from reports.models import SpecificStopSalesRules
from clients.models import Credential

from utils import DistributorApi

In [3]:
base_dir = Path(".").resolve()
data_dir = base_dir / "data"

In [3]:
credentials = Credential.objects.annotate(
    credential_id=F("id"), credential_name=F("name")
).values("credential_id", "credential_name", "enabled")


df_credentials = pd.DataFrame(credentials)

with pd.option_context("display.max_columns", None):
    display(df_credentials.head())

,credential_id,credential_name,is_enabled
0,32909,Nautalia_333_Zaragoza,True
1,34340,SUDAMERIA (ES),False
2,8726,MCC - Perfect Getaways - Birkenhead (Net),False
3,6165,No usar,False
4,34905,NEMO PLATFORM,True


In [4]:
stopsales_rules = (
    SpecificStopSalesRules.objects.filter(
        levels_rules__cli__t=1,
        tag__in=[-1, 1, 3],
        organization="lgt",
        active=True,
    )
    .annotate(credential_id=F("levels_rules__cli__l"))
    .values("rule_id", "active", "credential_id")
)

df_stopsales_rules = pd.DataFrame(list(stopsales_rules))
df_stopsales_rules = df_stopsales_rules.explode("credential_id").reset_index(
    drop=True
)

df_stopsales_rules = df_stopsales_rules.merge(
    df_credentials, on="credential_id", how="left"
)

In [5]:
with pd.option_context("display.max_columns", None):
    display(df_stopsales_rules.head())

,rule_id,active,credential_id,credential_name,is_enabled
0,509914,True,5906,Karavel,False
1,509914,True,5907,Karavel,False
2,509914,True,6008,Solbooking,False
3,509914,True,11240,Excite Exp,False
4,509914,True,4192,MyGo,False


In [6]:
df_result = (
    df_stopsales_rules.groupby(["rule_id", "active"])
    .agg(
        credential_count=("credential_id", "count"),
        enabled=("is_enabled", lambda x: (x == True).sum()),
        disabled=("is_enabled", lambda x: (x == False).sum()),
        credential_list=("credential_id", lambda x: list(set(x))),
        enabled_ids=(
            "credential_id",
            lambda x: list(
                set(x[df_stopsales_rules.loc[x.index, "is_enabled"]])
            ),
        ),
        disabled_ids=(
            "credential_id",
            lambda x: list(
                set(x[~df_stopsales_rules.loc[x.index, "is_enabled"]])
            ),
        ),
    )
    .reset_index()
)

df_result["disabled_percentage"] = (
    df_result["disabled"] / df_result["credential_count"]
)

df_result = df_result[df_result["disabled_percentage"] != 0]
df_result.sort_values(
    by=["disabled_percentage"], ascending=False, inplace=True
)

In [7]:
with pd.option_context("display.max_columns", None):
    display(df_result.head())

,rule_id,active,credential_count,enabled,disabled,credential_list,enabled_ids,disabled_ids,disabled_percentage
44,1655,True,10,1,9,"[4800, 5229, 4906, 5854, 8119, 8118, 9943, 412...",[4906],"[4800, 5229, 5854, 8119, 8118, 9943, 4120, 812...",0.900000
84,19075,True,7,1,6,"[4800, 11207, 5798, 9943, 6288, 6172, 10150]",[6288],"[4800, 11207, 5798, 9943, 6172, 10150]",0.857143
862,521073,True,7,1,6,"[4800, 12124, 11207, 33631, 12123, 12122, 9943]",[12122],"[4800, 12124, 11207, 33631, 12123, 9943]",0.857143
16,113557,True,6,1,5,"[12008, 4224, 11963, 12009, 5997, 12010]",[4224],"[12008, 11963, 12009, 5997, 12010]",0.833333
184,2789,True,4,1,3,"[9943, 11207, 5226, 4800]",[5226],"[9943, 11207, 4800]",0.750000


In [8]:
df_result.to_csv("stopsales_rules_inactive_credentials.csv", index=False)

In [9]:
api = DistributorApi()

In [10]:
rules = api.get_level_closes()

Rules retrieved successfully.


In [11]:
for index, row in df_result.iterrows():
    
    print(row.rule_id, row.enabled_ids)
    
    for rule in rules:
        if rule["id"] == row.rule_id:
            print(json.dumps(rule, indent=2))
            
            rule["lvl"]["cli"]["l"] = row.enabled_ids
            rule.pop("lastDate", None)
            
            print(json.dumps(rule, indent=2))
            
            break
    break

1655 ['4906']
{
  "editState": 1,
  "isObsolete": false,
  "name": "cierre B2C AC Group",
  "description": "Cierre b2c, ac group 2",
  "lastUser": "pau.cespedes@logitravelgroup.com",
  "lastDate": "21-09-2022 11:45:45",
  "tag": -1,
  "id": "1655",
  "lvl": {
    "rrg": null,
    "cli": {
      "t": 1,
      "l": [
        "4120",
        "4906",
        "4800",
        "5244",
        "5854",
        "8118",
        "8119",
        "8120",
        "9943",
        "5229"
      ]
    },
    "ps": {
      "t": 0,
      "l": null
    },
    "cp": 0,
    "rat": 1,
    "prv": {
      "t": 1,
      "l": [
        "ACG2"
      ]
    },
    "hot": {
      "t": 0,
      "l": null
    },
    "dest": {
      "t": 0,
      "l": null
    },
    "mrk": {
      "t": 0,
      "l": null
    },
    "mel": {
      "t": 0,
      "l": null
    },
    "cid": {
      "t": 0,
      "f": null,
      "u": null
    },
    "bod": {
      "t": 0,
      "f": null,
      "u": null
    },
    "rel": 0,
    "isDynamic

In [17]:
rule["id"] = "712860"
rule["name"] = "Test"
rule["description"] = "Test"
rule["lvl"]["cli"]["l"] = ['5226', '11730', '6288']
print(type(rule), rule)

<class 'dict'> {'editState': 1, 'isObsolete': False, 'name': 'Test', 'description': 'Test', 'tag': -1, 'id': '712860', 'lvl': {'rrg': None, 'cli': {'t': 1, 'l': ['5226', '11730', '6288']}, 'ps': {'t': 0, 'l': None}, 'cp': 0, 'rat': 1, 'prv': {'t': 1, 'l': ['ACG2']}, 'hot': {'t': 0, 'l': None}, 'dest': {'t': 0, 'l': None}, 'mrk': {'t': 0, 'l': None}, 'mel': {'t': 0, 'l': None}, 'cid': {'t': 0, 'f': None, 'u': None}, 'bod': {'t': 0, 'f': None, 'u': None}, 'rel': 0, 'isDynamicCommission': False, 'dow': {'t': 0, 'l': None}, 'age': 0, 'room': {'t': 0, 'l': None}, 'non': {'t': 0, 'l': None}, 'hou': {'t': 0, 'l': []}}}


In [15]:
new_rule = api.create_level_closes(rule)

Rule created successfully.


In [18]:
update_rule = api.update_level_closes(rule["id"], rule)

Rule updated successfully.
